### Read all FHIR SD and find all binding strengths

- load package data to inspect
- read SDs
- print binding by strength


In [77]:
from pathlib import Path
from json import load, dumps, loads
from IPython.display import display, Markdown, HTML 
table_header = '|Profile|Element|Value Set|Inherited from base?|\n|---|---|---|---|'
bindings = {
    'required':table_header,
    'extensible':table_header,
    'preferred':table_header,
    'example':table_header,
    'no_strength':table_header,
    
}


### get package

In [78]:
uscore_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev/package')
fhir_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')
print(uscore_path,fhir_path)

/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev/package /Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package


### Get SDs

In [79]:
def inherited(strength,element_path,sd_type):
    print(strength,element_path,sd_type)
    for f in fhir_path.iterdir():
        if str(f.name)==f'StructureDefinition-{sd_type}.json':
            print(f'### {f.name}')
            sd_dict=loads(f.read_text())
            for element in sd_dict['differential']['element']:
                if element.get('path')==element_path:
                    try:
                        base_strength=element['binding'].get('strength','no_strength')  
                    except KeyError:
                        base_strength="no binding!"
                    print(base_strength)
    try:
        return base_strength==strength
    except UnboundLocalError:
        return False

In [83]:
for f in uscore_path.iterdir():
    if str(f.name).startswith('StructureDefinition'):
        #print(f'### {f.name}')
        sd_dict=loads(f.read_text())
        #print(sd_dict['type'])
        for element in sd_dict['differential']['element']:
            if element.get('binding'):
                vs = element['binding'].get('valueSet','No ValueSet')
                vs = vs.split('|')[0] #strip version
                strength = element['binding'].get('strength','no_strength')
                print('foo')
                is_inherited = inherited(strength=strength,element_path=element["path"],sd_type=sd_dict['type'])
                bindings[strength]= f'{bindings[strength]}\n|{sd_dict["title"]}|{element["path"]}|{vs}|{f"**{is_inherited}**" if is_inherited == False else is_inherited}|'

foo
required Observation.status Observation
### StructureDefinition-Observation.json
required
foo
extensible Observation.code Observation
### StructureDefinition-Observation.json
example
foo
extensible Observation.dataAbsentReason Observation
### StructureDefinition-Observation.json
extensible
foo
required CareTeam.status CareTeam
### StructureDefinition-CareTeam.json
required
foo
extensible CareTeam.participant.role CareTeam
### StructureDefinition-CareTeam.json
example
foo
required Observation.valueQuantity.code Observation
### StructureDefinition-Observation.json
foo
required Patient.telecom.system Patient
### StructureDefinition-Patient.json
foo
required Patient.telecom.use Patient
### StructureDefinition-Patient.json
foo
required Patient.gender Patient
### StructureDefinition-Patient.json
required
foo
extensible Patient.address.state Patient
### StructureDefinition-Patient.json
foo
extensible Patient.communication.language Patient
### StructureDefinition-Patient.json
preferred
foo

In [85]:
for strength,binding in bindings.items():
    print(f'### {strength.upper()}')
    print(bindings[strength])
    #display(Markdown(f'### {strength.upper()}'))
    #display(Markdown(bindings[strength]))   

### REQUIRED
|Profile|Element|Value Set|Inherited from base?|
|---|---|---|---|
|US Core Laboratory Result Observation Profile|Observation.status|http://hl7.org/fhir/ValueSet/observation-status|True|
|US Core CareTeam Profile|CareTeam.status|http://hl7.org/fhir/ValueSet/care-team-status|True|
|US Core Body Temperature Profile|Observation.valueQuantity.code|http://hl7.org/fhir/ValueSet/ucum-bodytemp|**False**|
|US Core Patient Profile|Patient.telecom.system|http://hl7.org/fhir/ValueSet/contact-point-system|**False**|
|US Core Patient Profile|Patient.telecom.use|http://hl7.org/fhir/ValueSet/contact-point-use|**False**|
|US Core Patient Profile|Patient.gender|http://hl7.org/fhir/ValueSet/administrative-gender|True|
|US Core Immunization Profile|Immunization.status|http://hl7.org/fhir/ValueSet/immunization-status|True|
|US Core Smoking Status Observation Profile|Observation.status|http://hl7.org/fhir/us/core/ValueSet/us-core-observation-smoking-status-status|True|
|US Core DocumentReferenc